In [1]:
import os
import numpy as np
import netCDF4 as nc
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy
import xarray as xr
from datetime import datetime, timedelta
#from my_functions import sat_vap_press, vap_press, project_abi, grid_centers, contour_plot
import pandas as pd
import math

In [2]:
#IN: an nc dataset from GOES
#OUT LAT/LON in the right projection
def project_abi(data_cod): #grid edges
    # grab the constants needed to do the projection from GOES grid to lat/lon
    Req = data_cod.variables['goes_imager_projection'].semi_major_axis
    Rpol = data_cod.variables['goes_imager_projection'].semi_minor_axis
    lambda_0 = data_cod.variables['goes_imager_projection'].longitude_of_projection_origin
    lambda_0 = lambda_0*math.pi/180
    Hsat = data_cod.variables['goes_imager_projection'].perspective_point_height
    H = Hsat+Req
    
    # projection from GOES grid to lat/lon, following https://makersportal.com/blog/2018/11/25/goes-r-satellite-latitude-and-longitude-grid-projection-algorithm
    lat_rad_1d = data_cod['x'][:]
    lon_rad_1d = data_cod['y'][:]
    
    [X, Y] = np.meshgrid(lat_rad_1d, lon_rad_1d) # lon is x, lat is y, uses 'xy' indexing by default

    # sin and cos of x and y
    sinx = np.sin(X)
    sinx2 = sinx**2
    siny = np.sin(Y)
    siny2 = siny**2
    cosx = np.cos(X)
    cosx2 = cosx**2
    cosy = np.cos(Y)
    cosy2 = cosy**2

    a = np.power(np.sin(X),2.0) + (np.power(np.cos(X),2.0)*(np.power(np.cos(Y),2.0)+(((Req*Req)/(Rpol*Rpol))*np.power(np.sin(Y),2.0))))
    b = -2.0*H*np.cos(X)*np.cos(Y)
    c = (H**2.0)-(Req**2.0)

    # calculate rs
    rs = (-b - np.sqrt((b**2)-(4*a*c)))/2*a

    # calculate sx, sy, sz
    sx = rs*cosx*cosy
    sy = -rs*sinx
    sz = rs*cosx*siny
        
    # calculate lat and lon
    test = (Req**2/Rpol**2)*(sz/np.sqrt(((H-sx)**2)+(sy**2)))
    lat = np.arctan(test) 
    lon = lambda_0 - np.arctan(sy/(H-sx)) # in radians

    lat_deg = lat*180/math.pi
    lon_deg = lon*180/math.pi

    return lat_deg, lon_deg

In [42]:
#IN: a list of file names (FULL PATH), pre-projected lat and lon
#OUT: returns the stacked COD data to be merged/saved as a netcdf later
def stack_cod(fnames, lat_cod, lon_cod, mversion):
    datetimes = []
    #fill it up
    for ii in range(len(filenames)):
        # load the data
        name = fnames[ii].replace('M', mversion[ii])
        cod_data = nc.Dataset(name)
        #grab COD, lat and lon
        cod = cod_data['COD'][:,:].data
        cod[np.where(cod==np.amax(cod))] = 0 #set to 0 because we want to try max and mean aggregation. The value won't count for the mean and won't be looked at for the max
        #close
        cod_data.close()
        #make an empty stack
        cod_stack = np.zeros((len(filenames), lat_cod.shape[0], lat_cod.shape[1]))
        #add the current field to the stack
        cod_stack[ii, :,:] = cod
    
        # save the datestring associated with each file
        
        #get datestring: YYYYDDDHHMMSS.S
        d_str = name[56:70] #start times of the GOES-16 observations
        #transform into datetime object
        dtime_obj = datetime.strptime(d_str, "%Y%j%H%M%S%f")
        datetimes.append(dtime_obj)
    #save as an xarray dataset
    ds = xr.Dataset(
        {"cod": (('time', 'lat', 'lon'), cod_stack), 
         "lat_cod": (('lat', 'lon'), lat_cod), 
         "lon_cod": (('lat', 'lon'), lon_cod)}, 
        coords={
            "lat": range(lat_cod.shape[0]),
            "lon": range(lat_cod.shape[1]),
            "time": datetimes
            })
    return ds

In [3]:
data_cod = nc.Dataset('/data2/lthapa/2019/COD/201908615/OR_ABI-L2-CODC-M3_G16_s20190861502162_e20190861504535_c20190861506227.nc')
lat_cod, lon_cod = project_abi(data_cod)

/home/lthapa/anaconda3/envs/ML_py/lib/python3.7/site-packages/ipykernel_launcher.py:33: RuntimeWarning: invalid value encountered in sqrt
/home/lthapa/anaconda3/envs/ML_py/lib/python3.7/site-packages/ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in sqrt


In [5]:
print(lon_cod)

[[-- -- -- ... -52.96763610839844 -52.9321403503418 -52.896976470947266]
 [-- -- -- ... -52.989166259765625 -52.954345703125 -52.91872024536133]
 [-- -- -- ... -53.01118850708008 -52.976043701171875 -52.940452575683594]
 ...
 [-113.09178161621094 -113.06461334228516 -113.03719329833984 ...
  -61.94358825683594 -61.924034118652344 -61.90449905395508]
 [-113.0864028930664 -113.05924987792969 -113.03180694580078 ...
  -61.94504165649414 -61.925506591796875 -61.90595245361328]
 [-113.08116912841797 -113.05401611328125 -113.02656555175781 ...
  -61.94643020629883 -61.926876068115234 -61.907318115234375]]


In [45]:
prefix_goes ='/data2/lthapa/' # YR/YRDOYHR

start_date = datetime(2020, 1 , 1)
number_of_hours = (365+366)*24
date_list = [datetime.strftime((start_date + timedelta(hours = hour)),'%Y-%m-%d %H:%M:%S' ) for hour in range(number_of_hours)]
#print(date_list)
for item in date_list:
    year = datetime.strftime(datetime.strptime(item,'%Y-%m-%d %H:%M:%S'), '%Y')
    month = datetime.strftime(datetime.strptime(item,'%Y-%m-%d %H:%M:%S'), '%m')
    day = datetime.strftime(datetime.strptime(item,'%Y-%m-%d %H:%M:%S'), '%d')
    hour = datetime.strftime(datetime.strptime(item,'%Y-%m-%d %H:%M:%S'), '%H')
    minute = datetime.strftime(datetime.strptime(item,'%Y-%m-%d %H:%M:%S'), '%M')
    doy = datetime.strftime(datetime.strptime(item,'%Y-%m-%d %H:%M:%S'), '%j')

    path_goes = prefix_goes+year+'/COD/'+year+doy+hour+'/'
    print(path_goes)
    #get all the filenames for each hour
    filenames = []
    mver = []
    #grab all the files
    for root, dirs, files in os.walk(path_goes):
        for file in files:
            if 'M3' in file: #make sure all files have the same name
                file=file.replace('M3', 'M')
                mver.append('M3')
            elif 'M4' in file:
                file=file.replace('M4', 'M')
                mver.append('M4')
            elif 'M6' in file:
                file=file.replace('M6', 'M')
                mver.append('M6')
                
            filenames.append(path_goes+file)
    #zip and sort!
    zipped_lists = zip(filenames, mver)
    sorted_pairs = sorted(zipped_lists)
    #print(sorted_pairs)
    #tuples = zip(sorted_pairs)
    #filenames, mver = [ list(tuple) for tuple in  sorted_pairs]
    filenames = [tuple[0] for tuple in sorted_pairs]
    mver = [tuple[1] for tuple in sorted_pairs]

    if len(filenames)!=0:
        ds_stacked = stack_cod(filenames, lat_cod, lon_cod, mver)
    
        ds_mean_resample = ds_stacked.resample(time="H").mean()
        ds_max_resample = ds_stacked.resample(time="H").max()

        print(prefix_goes+year+'/'+'MEAN_COD_'+year+month+day+hour+'.nc', prefix_goes+year+'/'+'MAX_COD_'+year+month+day+hour+'.nc')
        ds_mean_resample.to_netcdf(path=prefix_goes+year+'/'+'MEAN_COD_'+year+month+day+hour+'.nc')
        ds_max_resample.to_netcdf(path=prefix_goes+year+'/'+'MAX_COD_'+year+month+day+hour+'.nc')
    else:
        print('MISSING: '+year+doy+hour)

/data2/lthapa/2020/COD/202000100/
/data2/lthapa/2020/MEAN_COD_2020010100.nc /data2/lthapa/2020/MAX_COD_2020010100.nc
/data2/lthapa/2020/COD/202000101/
/data2/lthapa/2020/MEAN_COD_2020010101.nc /data2/lthapa/2020/MAX_COD_2020010101.nc
/data2/lthapa/2020/COD/202000102/
/data2/lthapa/2020/MEAN_COD_2020010102.nc /data2/lthapa/2020/MAX_COD_2020010102.nc
/data2/lthapa/2020/COD/202000103/
/data2/lthapa/2020/MEAN_COD_2020010103.nc /data2/lthapa/2020/MAX_COD_2020010103.nc
/data2/lthapa/2020/COD/202000104/
/data2/lthapa/2020/MEAN_COD_2020010104.nc /data2/lthapa/2020/MAX_COD_2020010104.nc
/data2/lthapa/2020/COD/202000105/
/data2/lthapa/2020/MEAN_COD_2020010105.nc /data2/lthapa/2020/MAX_COD_2020010105.nc
/data2/lthapa/2020/COD/202000106/
/data2/lthapa/2020/MEAN_COD_2020010106.nc /data2/lthapa/2020/MAX_COD_2020010106.nc
/data2/lthapa/2020/COD/202000107/
/data2/lthapa/2020/MEAN_COD_2020010107.nc /data2/lthapa/2020/MAX_COD_2020010107.nc
/data2/lthapa/2020/COD/202000108/
/data2/lthapa/2020/MEAN_COD_20

/data2/lthapa/2020/MEAN_COD_2020010322.nc /data2/lthapa/2020/MAX_COD_2020010322.nc
/data2/lthapa/2020/COD/202000323/
/data2/lthapa/2020/MEAN_COD_2020010323.nc /data2/lthapa/2020/MAX_COD_2020010323.nc
/data2/lthapa/2020/COD/202000400/
/data2/lthapa/2020/MEAN_COD_2020010400.nc /data2/lthapa/2020/MAX_COD_2020010400.nc
/data2/lthapa/2020/COD/202000401/
/data2/lthapa/2020/MEAN_COD_2020010401.nc /data2/lthapa/2020/MAX_COD_2020010401.nc
/data2/lthapa/2020/COD/202000402/
/data2/lthapa/2020/MEAN_COD_2020010402.nc /data2/lthapa/2020/MAX_COD_2020010402.nc
/data2/lthapa/2020/COD/202000403/
/data2/lthapa/2020/MEAN_COD_2020010403.nc /data2/lthapa/2020/MAX_COD_2020010403.nc
/data2/lthapa/2020/COD/202000404/
/data2/lthapa/2020/MEAN_COD_2020010404.nc /data2/lthapa/2020/MAX_COD_2020010404.nc
/data2/lthapa/2020/COD/202000405/
/data2/lthapa/2020/MEAN_COD_2020010405.nc /data2/lthapa/2020/MAX_COD_2020010405.nc
/data2/lthapa/2020/COD/202000406/
/data2/lthapa/2020/MEAN_COD_2020010406.nc /data2/lthapa/2020/MAX

/data2/lthapa/2020/MEAN_COD_2020010621.nc /data2/lthapa/2020/MAX_COD_2020010621.nc
/data2/lthapa/2020/COD/202000622/
/data2/lthapa/2020/MEAN_COD_2020010622.nc /data2/lthapa/2020/MAX_COD_2020010622.nc
/data2/lthapa/2020/COD/202000623/
MISSING: 202000623
/data2/lthapa/2020/COD/202000700/
/data2/lthapa/2020/MEAN_COD_2020010700.nc /data2/lthapa/2020/MAX_COD_2020010700.nc
/data2/lthapa/2020/COD/202000701/
/data2/lthapa/2020/MEAN_COD_2020010701.nc /data2/lthapa/2020/MAX_COD_2020010701.nc
/data2/lthapa/2020/COD/202000702/
/data2/lthapa/2020/MEAN_COD_2020010702.nc /data2/lthapa/2020/MAX_COD_2020010702.nc
/data2/lthapa/2020/COD/202000703/
/data2/lthapa/2020/MEAN_COD_2020010703.nc /data2/lthapa/2020/MAX_COD_2020010703.nc
/data2/lthapa/2020/COD/202000704/
/data2/lthapa/2020/MEAN_COD_2020010704.nc /data2/lthapa/2020/MAX_COD_2020010704.nc
/data2/lthapa/2020/COD/202000705/
/data2/lthapa/2020/MEAN_COD_2020010705.nc /data2/lthapa/2020/MAX_COD_2020010705.nc
/data2/lthapa/2020/COD/202000706/
/data2/ltha

/data2/lthapa/2020/MEAN_COD_2020010920.nc /data2/lthapa/2020/MAX_COD_2020010920.nc
/data2/lthapa/2020/COD/202000921/
/data2/lthapa/2020/MEAN_COD_2020010921.nc /data2/lthapa/2020/MAX_COD_2020010921.nc
/data2/lthapa/2020/COD/202000922/
/data2/lthapa/2020/MEAN_COD_2020010922.nc /data2/lthapa/2020/MAX_COD_2020010922.nc
/data2/lthapa/2020/COD/202000923/
/data2/lthapa/2020/MEAN_COD_2020010923.nc /data2/lthapa/2020/MAX_COD_2020010923.nc
/data2/lthapa/2020/COD/202001000/
/data2/lthapa/2020/MEAN_COD_2020011000.nc /data2/lthapa/2020/MAX_COD_2020011000.nc
/data2/lthapa/2020/COD/202001001/
/data2/lthapa/2020/MEAN_COD_2020011001.nc /data2/lthapa/2020/MAX_COD_2020011001.nc
/data2/lthapa/2020/COD/202001002/
/data2/lthapa/2020/MEAN_COD_2020011002.nc /data2/lthapa/2020/MAX_COD_2020011002.nc
/data2/lthapa/2020/COD/202001003/
/data2/lthapa/2020/MEAN_COD_2020011003.nc /data2/lthapa/2020/MAX_COD_2020011003.nc
/data2/lthapa/2020/COD/202001004/
/data2/lthapa/2020/MEAN_COD_2020011004.nc /data2/lthapa/2020/MAX

/data2/lthapa/2020/MEAN_COD_2020011219.nc /data2/lthapa/2020/MAX_COD_2020011219.nc
/data2/lthapa/2020/COD/202001220/
/data2/lthapa/2020/MEAN_COD_2020011220.nc /data2/lthapa/2020/MAX_COD_2020011220.nc
/data2/lthapa/2020/COD/202001221/
/data2/lthapa/2020/MEAN_COD_2020011221.nc /data2/lthapa/2020/MAX_COD_2020011221.nc
/data2/lthapa/2020/COD/202001222/
/data2/lthapa/2020/MEAN_COD_2020011222.nc /data2/lthapa/2020/MAX_COD_2020011222.nc
/data2/lthapa/2020/COD/202001223/
/data2/lthapa/2020/MEAN_COD_2020011223.nc /data2/lthapa/2020/MAX_COD_2020011223.nc
/data2/lthapa/2020/COD/202001300/
/data2/lthapa/2020/MEAN_COD_2020011300.nc /data2/lthapa/2020/MAX_COD_2020011300.nc
/data2/lthapa/2020/COD/202001301/
/data2/lthapa/2020/MEAN_COD_2020011301.nc /data2/lthapa/2020/MAX_COD_2020011301.nc
/data2/lthapa/2020/COD/202001302/
/data2/lthapa/2020/MEAN_COD_2020011302.nc /data2/lthapa/2020/MAX_COD_2020011302.nc
/data2/lthapa/2020/COD/202001303/
/data2/lthapa/2020/MEAN_COD_2020011303.nc /data2/lthapa/2020/MAX

...

MISSING: 202119910
/data2/lthapa/2021/COD/202119911/
MISSING: 202119911
/data2/lthapa/2021/COD/202119912/
MISSING: 202119912
/data2/lthapa/2021/COD/202119913/
MISSING: 202119913
/data2/lthapa/2021/COD/202119914/
MISSING: 202119914
/data2/lthapa/2021/COD/202119915/
MISSING: 202119915
/data2/lthapa/2021/COD/202119916/
MISSING: 202119916
/data2/lthapa/2021/COD/202119917/
MISSING: 202119917
/data2/lthapa/2021/COD/202119918/
MISSING: 202119918
/data2/lthapa/2021/COD/202119919/
MISSING: 202119919
/data2/lthapa/2021/COD/202119920/
MISSING: 202119920
/data2/lthapa/2021/COD/202119921/
MISSING: 202119921
/data2/lthapa/2021/COD/202119922/
MISSING: 202119922
/data2/lthapa/2021/COD/202119923/
MISSING: 202119923
/data2/lthapa/2021/COD/202120000/
MISSING: 202120000
/data2/lthapa/2021/COD/202120001/
MISSING: 202120001
/data2/lthapa/2021/COD/202120002/
MISSING: 202120002
/data2/lthapa/2021/COD/202120003/
MISSING: 202120003
/data2/lthapa/2021/COD/202120004/
MISSING: 202120004
/data2/lthapa/2021/COD/2021

MISSING: 202123602
/data2/lthapa/2021/COD/202123603/
MISSING: 202123603
/data2/lthapa/2021/COD/202123604/
MISSING: 202123604
/data2/lthapa/2021/COD/202123605/
MISSING: 202123605
/data2/lthapa/2021/COD/202123606/
MISSING: 202123606
/data2/lthapa/2021/COD/202123607/
MISSING: 202123607
/data2/lthapa/2021/COD/202123608/
MISSING: 202123608
/data2/lthapa/2021/COD/202123609/
MISSING: 202123609
/data2/lthapa/2021/COD/202123610/
MISSING: 202123610
/data2/lthapa/2021/COD/202123611/
MISSING: 202123611
/data2/lthapa/2021/COD/202123612/
MISSING: 202123612
/data2/lthapa/2021/COD/202123613/
MISSING: 202123613
/data2/lthapa/2021/COD/202123614/
MISSING: 202123614
/data2/lthapa/2021/COD/202123615/
MISSING: 202123615
/data2/lthapa/2021/COD/202123616/
MISSING: 202123616
/data2/lthapa/2021/COD/202123617/
MISSING: 202123617
/data2/lthapa/2021/COD/202123618/
MISSING: 202123618
/data2/lthapa/2021/COD/202123619/
MISSING: 202123619
/data2/lthapa/2021/COD/202123620/
MISSING: 202123620
/data2/lthapa/2021/COD/2021

MISSING: 202127223
/data2/lthapa/2021/COD/202127300/
MISSING: 202127300
/data2/lthapa/2021/COD/202127301/
MISSING: 202127301
/data2/lthapa/2021/COD/202127302/
MISSING: 202127302
/data2/lthapa/2021/COD/202127303/
MISSING: 202127303
/data2/lthapa/2021/COD/202127304/
MISSING: 202127304
/data2/lthapa/2021/COD/202127305/
MISSING: 202127305
/data2/lthapa/2021/COD/202127306/
MISSING: 202127306
/data2/lthapa/2021/COD/202127307/
MISSING: 202127307
/data2/lthapa/2021/COD/202127308/
MISSING: 202127308
/data2/lthapa/2021/COD/202127309/
MISSING: 202127309
/data2/lthapa/2021/COD/202127310/
MISSING: 202127310
/data2/lthapa/2021/COD/202127311/
MISSING: 202127311
/data2/lthapa/2021/COD/202127312/
MISSING: 202127312
/data2/lthapa/2021/COD/202127313/
MISSING: 202127313
/data2/lthapa/2021/COD/202127314/
MISSING: 202127314
/data2/lthapa/2021/COD/202127315/
MISSING: 202127315
/data2/lthapa/2021/COD/202127316/
MISSING: 202127316
/data2/lthapa/2021/COD/202127317/
MISSING: 202127317
/data2/lthapa/2021/COD/2021

MISSING: 202130915
/data2/lthapa/2021/COD/202130916/
MISSING: 202130916
/data2/lthapa/2021/COD/202130917/
MISSING: 202130917
/data2/lthapa/2021/COD/202130918/
MISSING: 202130918
/data2/lthapa/2021/COD/202130919/
MISSING: 202130919
/data2/lthapa/2021/COD/202130920/
MISSING: 202130920
/data2/lthapa/2021/COD/202130921/
MISSING: 202130921
/data2/lthapa/2021/COD/202130922/
MISSING: 202130922
/data2/lthapa/2021/COD/202130923/
MISSING: 202130923
/data2/lthapa/2021/COD/202131000/
MISSING: 202131000
/data2/lthapa/2021/COD/202131001/
MISSING: 202131001
/data2/lthapa/2021/COD/202131002/
MISSING: 202131002
/data2/lthapa/2021/COD/202131003/
MISSING: 202131003
/data2/lthapa/2021/COD/202131004/
MISSING: 202131004
/data2/lthapa/2021/COD/202131005/
MISSING: 202131005
/data2/lthapa/2021/COD/202131006/
MISSING: 202131006
/data2/lthapa/2021/COD/202131007/
MISSING: 202131007
/data2/lthapa/2021/COD/202131008/
MISSING: 202131008
/data2/lthapa/2021/COD/202131009/
MISSING: 202131009
/data2/lthapa/2021/COD/2021

MISSING: 202134611
/data2/lthapa/2021/COD/202134612/
MISSING: 202134612
/data2/lthapa/2021/COD/202134613/
MISSING: 202134613
/data2/lthapa/2021/COD/202134614/
MISSING: 202134614
/data2/lthapa/2021/COD/202134615/
MISSING: 202134615
/data2/lthapa/2021/COD/202134616/
MISSING: 202134616
/data2/lthapa/2021/COD/202134617/
MISSING: 202134617
/data2/lthapa/2021/COD/202134618/
MISSING: 202134618
/data2/lthapa/2021/COD/202134619/
MISSING: 202134619
/data2/lthapa/2021/COD/202134620/
MISSING: 202134620
/data2/lthapa/2021/COD/202134621/
MISSING: 202134621
/data2/lthapa/2021/COD/202134622/
MISSING: 202134622
/data2/lthapa/2021/COD/202134623/
MISSING: 202134623
/data2/lthapa/2021/COD/202134700/
MISSING: 202134700
/data2/lthapa/2021/COD/202134701/
MISSING: 202134701
/data2/lthapa/2021/COD/202134702/
MISSING: 202134702
/data2/lthapa/2021/COD/202134703/
MISSING: 202134703
/data2/lthapa/2021/COD/202134704/
MISSING: 202134704
/data2/lthapa/2021/COD/202134705/
MISSING: 202134705
/data2/lthapa/2021/COD/2021

In [ ]:
20203662301176